# 라이브러리 설치

In [1]:
!pip install --upgrade --quiet langchain langchain-openai langchainhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00


# LangSmith 설정 (Optional)

In [22]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangChain Tool Use Quickstart - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "your_langsmith_key"

In [3]:
unique_id

'e3549286'

# Create a tool

In [4]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
  """Multiply two integers together."""
  return first_int * second_int

In [5]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [6]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

#Chain

In [8]:
OPENAI_KEY = "your_OpenAI_key"

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=OPENAI_KEY)

In [10]:
llm_with_tools = llm.bind_tools([multiply])

In [11]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_36xGNcR3wqjB47pDjiaSk3zE',
  'type': 'tool_call'}]

# Invoking the tool

In [12]:
from operator import itemgetter
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("whats 5 times 23")

115

# Agent(중요!)

In [13]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [14]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [15]:
@tool
def add(first_int: int, second_int: int) -> int:
  "Add two integers."
  return first_int + second_int

In [16]:
@tool
def exponentiate(base: int, exponent: int) -> int:
  "Exponentiate the base to the exponent power."
  return base**exponent

In [18]:
tools = [multiply, add, exponentiate]

In [19]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [20]:
agent_excutor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
agent_excutor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of taking \(3\) to the fifth power and multiplying that by the sum of \(12\) and \(3\), and then squaring the whole result is \(13,286,025\).

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of taking \\(3\\) to the fifth power and multiplying that by the sum of \\(12\\) and \\(3\\), and then squaring the whole result is \\(13,286,025\\).'}